In [183]:
import re

#### 1-Eliminate implication

In [485]:
# Initial setup
counter = 97
# Define the replace_char_at_index function
def index_char(str, indx, char_new):
    return str[:indx] + char_new + str[indx + 1:]

# Define the eliminate_implication function
def eliminate_implication(formula):
    for i in range(len(formula)):
        if formula[i] == '⇒':
            # Replace '⇒' with '∨'
            formula = index_char(formula, i, "∨")
            for j in range(i-1, -1, -1):
                if formula[j] == '(':
                    # Replace the character before '(' with '~'
                    formula = formula.replace(formula[j-1], "~" + formula[j-1])
                    break
    return formula


# Test the function
logical_expression = "∀x(P(x)⇒Q(x))⇒(∀x z(x)⇒∀x s(x))"
expression_without_double_negation = eliminate_implication(logical_expression)
print("Original expression:", logical_expression)
print("Expression without double negation:", expression_without_double_negation)



Original expression: ∀x(P(x)⇒Q(x))⇒(∀x z(x)⇒∀x s(x))
Expression without double negation: ∀x(~P(x)∨~Q(x))∨(∀x ~z(x)∨∀x s(x))


####  3-Remove double-not.

In [440]:
def remove_doube(exp):
  result = []
  i = 0
  while i < len(exp):
    # Check for double negation (~~)
    if exp[i:i+2] == '~~':
      # Skip over the double negation
      i += 2
    else:
      # Append the character to the result
      result.append(exp[i])
      i += 1
  return ''.join(result)

# Example usage
logical_expression = "~~(P & Q) "
expression_without_double_negation = remove_doube(logical_expression)
print("Original expression:", logical_expression)
print("Expression without double negation:", expression_without_double_negation)


Original expression: ['~', '~', ['&', 'P', 'Q']]
Expression without double negation: [['&', 'P', 'Q']]


#### 2-Move negation inward (Demorgan Law)

In [443]:
def if_quantis(exp):
    final_expression = ""
    counter = 0
    # Loop through each character in the expression
    while counter < len(exp):
        # check if the current position start with negation forall  or negation there exist 
        if exp.startswith("~∀", counter) or exp.startswith("~∃", counter):
            # Extract the quantifier type (opposite of the negated one)
            quanti = '∃' if exp[counter + 1] == '∀' else '∀'
            # Add the extracted quantifier to the final expression
            final_expression += quanti
            # Move the counter past the negation and quantifier symbols
            counter += 2
            # Add the next character (variable name) and a negation symbol
            variable = exp[counter]
            final_expression += variable + "~"
            counter += 1
        else:
            # If not a negated quantifier, simply add the current character
            final_expression += exp[counter]
            counter += 1
    return final_expression

def MNI(expression):
    if expression.find('~') == 2:
        expression = remove_doube(expression)
    # Apply quantifier manipulation
    expression = if_quantis(expression)
    # Move negation inward using De Morgan's Law
    # Define patterns for negation of AND and OR
    pat_and = r'~\((.*?)\s*&\s*(.*?)\)'
    pat_or = r'~\((.*?)\s*\|\s*(.*?)\)'
     # Apply De Morgan's Law for AND and OR
    expression = re.sub(pat_and, r'(~\1 | ~\2)', expression)
    expression = re.sub(pat_or, r'(~\1 & ~\2)', expression)
    
    # Check if the expression starts with a negation
    if expression.startswith('~'):
        # Remove the negation symbol (~) from the expression
        expression = expression[1:]
        # Negate the entire expression
        expression = "~(" + expression + ")"
    
    # Loop until all negations (~) have been moved inward
    while True:
        # Find the start index of the next negation (~)
        negation_start = expression.find('~(')
        # If no negation is found, exit the loop
        if negation_start == -1:
            break
        
        # Find the corresponding closing parenthesis for the negation
        open_paren_count = 1
        negation_end = negation_start + 2
        for i in range(negation_start + 2, len(expression)):
            if expression[i] == '(':
                open_paren_count += 1
            elif expression[i] == ')':
                open_paren_count -= 1
                if open_paren_count == 0:
                    negation_end = i
                    break
        
        # Extract the inner formula enclosed by the negation
        inner_formula = expression[negation_start + 2:negation_end]
        
        # Split the inner formula by the connective symbols (| or &)
        subExp = inner_formula.split('|') if '|' in inner_formula else inner_formula.split('&')
        
        # Negate each subformula and reconstruct the inner formula
        negated_subformulas = ['~' + subformula if subformula[0] != '~' else subformula[1:] for subformula in subExp]
        # Determine the new logical connective based on the original inner formula
        new_connective = '&' if '|' in inner_formula else '|'
        inner_formula = '(' + new_connective.join(negated_subformulas) + ')'
        
        # Replace the negated expression with the transformed inner formula
        expression = expression[:negation_start] + inner_formula + expression[negation_end + 1:]
    
    # Return the expression with all negations moved inward
    return expression

# Example usage:
formula = "ّّ~~∀x(~(P(x) & Q(x)) | ~R(x))"
formula1 = "~∀x(P(x) & Q(x))"
moved_formula = MNI(formula)
print("Original formula:", formula)
print("Formula with negation moved inward:", moved_formula)
print("=======================================")
moved_formula1 = MNI(formula1)
print("Original formula1:", formula1)
print("Formula with negation moved inward:", moved_formula1)
print("=======================================")
formula2 = "~∃x(~P(x) | Q(x))"
moved_formula2 = MNI(formula2)
print("Original formula2:", formula2)
print("Formula with negation moved inward:", moved_formula2)
print("=======================================")
formula3 = "~∃x∀y(P(x, y) & Q(x, y))"
moved_formula3 = MNI(formula3)
print("Original formula3:", formula3)
print("Formula with negation moved inward:", moved_formula3)
print("=======================================")




Original expression: =>&PQ=>~R|ST
Expression after Multi-Not Inward (MNI) transformation: =>&PQ=>~R|ST


####  4-Standardize variable scope

In [452]:

def SVS(exp):
    result = ""
    for expr in exp:
        new_expr = expr
        vars_seen = set()
        rep_map = {}
        
        def var_new(var_old):
            if var_old in rep_map:
                return rep_map[var_old]
            new_var = chr(ord('a') + len(vars_seen))
            while new_var in vars_seen:
                new_var = chr(ord(new_var) + 1)
            vars_seen.add(new_var)
            rep_map[var_old] = new_var
            return new_var

        counter = 0
        while counter < len(new_expr):
            if new_expr[counter] in "∀∃":
                if counter + 1 < len(new_expr):
                    var = new_expr[counter + 1]
                    if var.islower():  
                        new_var = var_new(var)
                        new_expr = new_expr[:counter + 1] + new_expr[counter + 1:].replace(var, new_var)
                        counter += len(new_var) - 1
            counter += 1
        result += new_expr   # Concatenate the processed expression with a space
        
    return result.strip()  # Remove trailing space before returning

# Test case

expressions_1 = ['∃x(P(x) & Q(x)) '  ' ∀x(R(x) | S(x))']
print("Original Expressions 1:", expressions_1)
print("SVS Result 1:", SVS(expressions_1))


# Test case 3
expressions_2 = ['∃x∀y(P(x, y) & Q(x, y))' '∀z∃w(R(z, w) | S(z, w))']
print("\nOriginal Expressions 3:", expressions_2)
print("SVS Result 3:", SVS(expressions_2))


Original Expressions 1: ['∃x(P(x) & Q(x))  ∀x(R(x) | S(x))']
SVS Result 1: ∃a(P(a) & Q(a))  ∀b(R(b) | S(b))

Original Expressions 3: ['∃x∀y(P(x, y) & Q(x, y))∀z∃w(R(z, w) | S(z, w))']
SVS Result 3: ∃a∀b(P(a, b) & Q(a, b))∀c∃d(R(c, d) | S(c, d))


#### 5- The prenex form (obtained by moving all quantifiers to the left of the
formula.)

In [66]:
#done
def penex_form(exp):
    quanti_part = ""
    parts = ""
    i = 0
    while i < len(exp):
        if exp[i] == '∀' or exp[i] == '∃':
            quanti_part += exp[i] + exp[i+1] + ' '
            i += 1  # Skip the next character which is the variable
        else:
            parts += exp[i]
        i += 1
    return quanti_part.strip() + " " + parts.strip()

# Test case
expression = '∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))'
print("Original Expression:", expression)
print("Penex Form:", penex_form(expression))


Original Expression: ∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))
Penex Form: ∃x ∀y (P(x) ∧ Q(x)) ∧ (R(y) ∨ S(y))


#### 6-Skolemization for existential quantifiers

In [226]:
def skolemization(exp):
    part = exp.split('∃')
    parts_skol = [part[0]]
    for part in part[1:]:
        if part.strip():
            var, i, exp_in = part.partition('(')
            var += '('  # Adding back the opening parenthesis
            
            # Create Skolem function with parentheses
            func_skol = "f" + "("+var.replace('(', '').replace(')', ')')  + ")" # Add closing parenthesis
            
            # Replace variable x in the inner expression with the Skolem function
            skolemized_inner_exp = exp_in.replace('x',  func_skol )
            
            
            # Append skolemized inner expression to the list of skolemized parts
            parts_skol.append(skolemized_inner_exp)
    return ''.join(parts_skol)

# Example usage:
logical_expression = "∃x (P(x) & Q(x))"
skolemized_expression = skolemization(logical_expression)
print("Original expression:", logical_expression)
print("Skolemized expression:", skolemized_expression)


Original expression: ∃x (P(x) & Q(x))
Skolemized expression: P(f(x )) & Q(f(x )))


#### 7-Eliminate universal quantifiers.

In [299]:
# done
def E_U(exp):
    # Define a pattern to match universal quantifiers and their corresponding variables and inner expressions
    patrn = r'∀(\w+)\s*\((.*?)\)'

    # Use regex to remove all occurrences of universal quantifiers
    withoutForAll = re.sub(patrn, r'(\2)', exp)

    return withoutForAll

# Example usage:
logical_expression = "∀x (P(x) & Q(x))"
expression_without_universal_quantifiers = E_U(logical_expression)
print("Original expression:", logical_expression)
print("Expression without universal quantifiers:", expression_without_universal_quantifiers)


Original expression: ∀x (P(x) & Q(x))
Expression without universal quantifiers: (P(x) & Q(x))


#### 8-Convert to conjunctive normal form

In [464]:
import re

def eliminate_bidirectional(expression):
    # Define a pattern to match bidirectional implications
    pat = r'(\b\w+\b)\s*<=>\s*(\b\w+\b)'
    
    # Replace each bidirectional implication with the conjunction of implications
    while re.search(pat, expression):
        expression = re.sub(pat, r'(\1 -> \2) & (\2 -> \1)', expression)
    
    return expression

# Test cases
test_cases = [
    "(P <=> Q)",                             # Simple bidirectional implication
    "(P <=> Q) <=> (R <=> S)",               # Multiple bidirectional implications
    "(A <=> B) <=> (C <=> D) <=> (E <=> F)", # Multiple bidirectional implications
    "(P <=> Q) & (R <=> S)",                 # Bidirectional implications with other operators
    "(P <=> Q) & (R <=> S) & (T <=> U)"      # Bidirectional implications with other operators and multiples
]

print("Original expressions and their equivalents after bidirectional elimination:")
for idx, expression in enumerate(test_cases, 1):
    result = eliminate_bidirectional(expression)
    print(f"Test case {idx}:")
    print("Original expression:", expression)
    print("Expression after bidirectional elimination:", result)
    print()


Original expressions and their equivalents after bidirectional elimination:
Test case 1:
Original expression: (P <=> Q)
Expression after bidirectional elimination: ((P -> Q) & (Q -> P))

Test case 2:
Original expression: (P <=> Q) <=> (R <=> S)
Expression after bidirectional elimination: ((P -> Q) & (Q -> P)) <=> ((R -> S) & (S -> R))

Test case 3:
Original expression: (A <=> B) <=> (C <=> D) <=> (E <=> F)
Expression after bidirectional elimination: ((A -> B) & (B -> A)) <=> ((C -> D) & (D -> C)) <=> ((E -> F) & (F -> E))

Test case 4:
Original expression: (P <=> Q) & (R <=> S)
Expression after bidirectional elimination: ((P -> Q) & (Q -> P)) & ((R -> S) & (S -> R))

Test case 5:
Original expression: (P <=> Q) & (R <=> S) & (T <=> U)
Expression after bidirectional elimination: ((P -> Q) & (Q -> P)) & ((R -> S) & (S -> R)) & ((T -> U) & (U -> T))



In [297]:
def to_conj_of_disj(expression):
    # Split the expression by '&' to get individual conjuncts
    conjuncts = expression.split('&')
    
    # Initialize the result as an empty string
    result = ""
    
    # Iterate through each conjunct
    for conjunct in conjuncts:
        # If the conjunct contains '|', split it by '|' to get individual disjunctions
        if '|' in conjunct:
            disjunctions = conjunct.split('|')
            # If there are multiple disjunctions, wrap them in parentheses
            if len(disjunctions) > 1:
                conjunct = '(' + conjunct + ')'
        
        # Append the modified conjunct to the result with '&' in between
        result += conjunct + ' & '
    
    # Remove the trailing '&' and return the result
    return result[:-3]  # Remove the last ' & '

# Test case for to_conj_of_disj function
expression = "(p & q) | r"
result = to_conj_of_disj(expression)
print("Original expression:", expression)
print("Expression in conjunctive form:", result)
print("===========================================")
# Test case 2
expression_2 = "(p & q) | (r | s)"
result_2 = to_conj_of_disj(expression_2)
print("\nOriginal expression 2:", expression_2)
print("Expression in conjunctive form 2:", result_2)
print("===========================================")
# Test case 3
expression_3 = "(p & q) | (r & s)"
result_3 = to_conj_of_disj(expression_3)
print("\nOriginal expression 3:", expression_3)
print("Expression in conjunctive form 3:", result_3)
# Test case 4
print("===========================================")
expression_4 = "p & (q | r) & (s | t)"
result_4 = to_conj_of_disj(expression_4)
print("\nOriginal expression 4:", expression_4)
print("Expression in conjunctive form 4:", result_4)

Original expression: (p & q) | r
Expression in conjunctive form: (p  & ( q) | r)

Original expression 2: (p & q) | (r | s)
Expression in conjunctive form 2: (p  & ( q) | (r | s))

Original expression 3: (p & q) | (r & s)
Expression in conjunctive form 3: (p  & ( q) | (r ) &  s)

Original expression 4: p & (q | r) & (s | t)
Expression in conjunctive form 4: p  & ( (q | r) ) & ( (s | t))


In [465]:
def convert_to_CNF(expression):
    print("Original Expression:", expression)
    
    # Step 1: Eliminate BIDIRECTIONAL
    expression = eliminate_bidirectional(expression)
    print("After Step 1 (Eliminate BIDIRECTIONAL):", expression)
    
    # Step 2: Eliminate Implications
    expression = eliminate_implication(expression)
    print("After Step 2 (Eliminate Implications):", expression)
    
    # Step 3: Move Negation Inward
    expression = MNI(expression)
    print("After Step 3 (Move Negation Inward):", expression)
    
    # Step 4: Remove Double Negations
    expression = remove_doube(expression)
    print("After Step 4 (Remove Double Negations):", expression)
    
    # Step 5: Standardize Variable Scope
    expression = [expression]
    
    expression = SVS(expression)
    print("After Step 5 (Standardize Variable Scope):", expression)
    # Step 6: Move Universal Quantifiers Outward
    expression = E_U(expression)
    print("After Step 6 (Move Universal Quantifiers Outward):", expression)
    
    # Step 7: Skolemization
    expression = skolemization(expression)
    print("After Step 7 (Skolemization):", expression)
    
    # Step 8: Convert to CNF
    expression = to_conj_of_disj(expression)
    print("After Step 8 (Convert to CNF):", expression)
    
    return expression

# Test cases
logical_expression_1 = "∀x(P(x)⇒Q(x))"
cnf_expression_1 = convert_to_CNF(logical_expression_1)
print("\nFinal CNF Expression 1:", cnf_expression_1)
print("===============================================")

logical_expression_2 = "(P<=>Q)"
cnf_expression_2 = convert_to_CNF(logical_expression_2)
print("\nFinal CNF Expression 2:", cnf_expression_2)
print("===============================================")




Original Expression: ∀x(P(x)⇒Q(x))
After Step 1 (Eliminate BIDIRECTIONAL): ∀x(P(x)⇒Q(x))
After Step 2 (Eliminate Implications): ∀x(¬P(x)∨Q(x))
After Step 3 (Move Negation Inward): ∀x(¬P(x)∨Q(x))
After Step 4 (Remove Double Negations): ∀x(¬P(x)∨Q(x))
After Step 5 (Standardize Variable Scope): ∀a(¬P(a)∨Q(a))
After Step 6 (Move Universal Quantifiers Outward): (¬P(a)∨Q(a))
After Step 7 (Skolemization): (¬P(a)∨Q(a))
After Step 8 (Convert to CNF): (¬P(a)∨Q(a))

Final CNF Expression 1: (¬P(a)∨Q(a))
Original Expression: (P<=>Q)
After Step 1 (Eliminate BIDIRECTIONAL): ((P -> Q) & (Q -> P))
After Step 2 (Eliminate Implications): ((P -> Q) & (Q -> P))
After Step 3 (Move Negation Inward): ((P -> Q) & (Q -> P))
After Step 4 (Remove Double Negations): ((P -> Q) & (Q -> P))
After Step 5 (Standardize Variable Scope): ((P -> Q) & (Q -> P))
After Step 6 (Move Universal Quantifiers Outward): ((P -> Q) & (Q -> P))
After Step 7 (Skolemization): ((P -> Q) & (Q -> P))
After Step 8 (Convert to CNF): ((P -> Q)

In [469]:
# Step 9: Turn Conjunctions into Clauses
def turn_conjunctions_into_clauses(expression):
    # Split the expression by '&' to get individual conjuncts
    conjuncts = expression.split('&')
    
    # Initialize the list to store clauses
    clauses = []
    
    # Iterate through each conjunct
    for conjunct in conjuncts:
        # If the conjunct contains '|', split it by '|' to get individual disjunctions
        if '|' in conjunct:
            disjunctions = conjunct.split('|')
            # If there are multiple disjunctions, wrap them in parentheses
            if len(disjunctions) > 1:
                conjunct = '(' + conjunct + ')'
        
        # Append the modified conjunct to the list of clauses
        clauses.append(conjunct.strip())
    
    return set(clauses)  # Using set to ensure uniqueness of clauses

In [470]:
# Define the function
def turn_conjunctions_into_clauses(expression):
    # Split the expression by '&' to get individual conjuncts
    conjuncts = expression.split('&')
    
    # Initialize the list to store clauses
    clauses = []
    
    # Iterate through each conjunct
    for conjunct in conjuncts:
        # If the conjunct contains '|', split it by '|' to get individual disjunctions
        if '|' in conjunct:
            disjunctions = conjunct.split('|')
            # If there are multiple disjunctions, wrap them in parentheses
            if len(disjunctions) > 1:
                conjunct = '(' + conjunct + ')'
        
        # Append the modified conjunct to the list of clauses
        clauses.append(conjunct.strip())
    
    return set(clauses)  # Using set to ensure uniqueness of clauses

# Test case
expression = "(A & B) | (C & D) | E"
result = turn_conjunctions_into_clauses(expression)
print("Original Expression:", expression)
print("Clauses:", result)


Original Expression: (A & B) | (C & D) | E
Clauses: {'(A', '( D) | E)', '( B) | (C )'}


In [ ]:
# Apply Resolution Procedure
def resolution_procedure(expression):
    # Apply each step of resolution procedure
    expression = eliminate_implication(expression)
    expression = MNI(expression)
    expression = remove_doube(expression)
    expression = SVS(expression)
    expression = penex_form(expression)
    expression = skolemization(expression)
    expression = E_U(expression)
    expression = convert_to_CNF(expression)
    #note complete 
    # expression = turn_conjunctions_into_clauses(expression)
    # expression = rename_variables_in_clauses(expression)
    
    # Perform resolution here
    
    return expression